In [1]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.stores import InMemoryByteStore
from langchain_classic.retrievers import ParentDocumentRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [5]:
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  
    max_tokens=200,             
)

In [6]:
pdf_path = "os-sertoes.pdf"
load_pdf = PyPDFLoader(pdf_path, extract_images=False)
pages = load_pdf.load_and_split()

In [7]:
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200  
)

parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,          
    chunk_overlap=200,        
    length_function=len,      
    add_start_index=True      
)

In [16]:
store = InMemoryByteStore()
vectorstore = Chroma(embedding_function=embeddings_model)

In [11]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,       
    docstore=store,                
    child_splitter=child_splitter, 
    parent_splitter=parent_splitter 
)

batch_size = 50
for i in range(0, len(pages), batch_size):
    batch = pages[i:i + batch_size]
    parent_document_retriever.add_documents(batch, ids=None)
    print(f"Batch {i // batch_size + 1}/{(len(pages) + batch_size - 1) // batch_size} adicionado ({len(batch)} páginas)")

Batch 1/14 adicionado (50 páginas)
Batch 2/14 adicionado (50 páginas)
Batch 3/14 adicionado (50 páginas)
Batch 4/14 adicionado (50 páginas)
Batch 5/14 adicionado (50 páginas)
Batch 6/14 adicionado (50 páginas)
Batch 7/14 adicionado (50 páginas)
Batch 8/14 adicionado (50 páginas)
Batch 9/14 adicionado (50 páginas)
Batch 10/14 adicionado (50 páginas)
Batch 11/14 adicionado (50 páginas)
Batch 12/14 adicionado (50 páginas)
Batch 13/14 adicionado (50 páginas)
Batch 14/14 adicionado (8 páginas)


In [12]:
TEMPLATE = """
Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado:

Contexto: {context}

Pergunta: {question}
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=TEMPLATE)

parser = StrOutputParser()

sequence = prompt | llm | parser

In [13]:
def answer_question(question: str):
    context = parent_document_retriever.invoke(question)

    response = sequence.invoke({"context": context, "question": question})

    return response

In [14]:
for index, question in enumerate(questions):
    resposta = answer_question(question)
    print({"numero": index, "pergunta": question, "resposta": resposta})

{'numero': 0, 'pergunta': 'Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?', 'resposta': 'Euclides da Cunha descreve o ambiente natural do sertão nordestino como um lugar árido, marcado pela seca e pela escassez de recursos hídricos. Ele destaca a dureza da região, caracterizada pela falta de chuvas e pela presença de terras ressequidas. Essas condições adversas tornam a vida dos habitantes do sertão um verdadeiro martírio, refletindo uma tortura maior que abrange a economia geral da vida. Euclides da Cunha mostra como o ambiente natural hostil do sertão nordestino influencia diretamente a vida das pessoas que ali habitam, provocando um sofrimento profundo e uma luta constante pela sobrevivência em meio às dificuldades impostas pela seca e pela aridez do solo.'}
{'numero': 1, 'pergunta': 'Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas 